In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1,org.apache.spark:spark-avro_2.12:3.3.1 pyspark-shell'

In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark = SparkSession \
    .builder \
    .appName("Spark-Notebook") \
    .getOrCreate()

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5323724d-b4cd-4491-b364-0f92c225b8d6;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.1 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.common

24/02/21 10:14:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
# default for startingOffsets is "latest"
df_kafka_raw = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092,broker:29092") \
    .option("subscribe", "quickstart-events") \
    .option("startingOffsets", "earliest") \
    .option("checkpointLocation", "checkpoint") \
    .load()

In [4]:
df_kafka_raw.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [5]:
#1.2 Encoded Kafka Stream

df_kafka_encoded = df_kafka_raw.selectExpr("CAST(key AS STRING)","CAST(value AS STRING)")

In [6]:
df_kafka_encoded.printSchema()

root
 |-- key: string (nullable = true)
 |-- value: string (nullable = true)



# Mahesh Start

In [7]:
from pyspark.sql.functions import from_json
from pyspark.sql.types import StructType, StructField, StringType

structType_schema = StructType(
    [StructField("name", StringType()),
     StructField('address', StringType()),
     StructField('created_at', StringType()),
     ])

df1 = df_kafka_encoded.withColumn('valueStruct', from_json(df_kafka_encoded.value, structType_schema))

df1.printSchema()

root
 |-- key: string (nullable = true)
 |-- value: string (nullable = true)
 |-- valueStruct: struct (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- address: string (nullable = true)
 |    |-- created_at: string (nullable = true)



In [8]:
df2 = df1.withColumn('name',df1['valueStruct']['name']) \
    .withColumn('address', df1['valueStruct']['address'] ) \
    .withColumn('created_at', df1['valueStruct']['created_at'] )
df2.printSchema()

root
 |-- key: string (nullable = true)
 |-- value: string (nullable = true)
 |-- valueStruct: struct (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- address: string (nullable = true)
 |    |-- created_at: string (nullable = true)
 |-- name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- created_at: string (nullable = true)



In [ ]:
# df = df.select("col1","col2","col3").withColumn("col4", df['col1'])

In [9]:
df3 = df2.select("name","address","created_at")

df3.printSchema()

root
 |-- name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- created_at: string (nullable = true)



In [10]:
def sink_console(df, output_mode: str = 'complete', processing_time: str = '4 seconds'):
    write_query = df.writeStream \
        .outputMode(output_mode) \
        .trigger(processingTime=processing_time) \
        .format("console") \
        .option("truncate", False) \
        .start()
    return write_query # pyspark.sql.streaming.StreamingQuery

In [11]:
write_query = sink_console(df3, output_mode='append')

24/02/21 10:16:11 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-e6201022-f113-4745-b8b4-f3120966024a. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/02/21 10:16:11 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/02/21 10:16:11 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-936e407b-2088-489d-8598-4fa287ce9ae7--1600603854-driver-0-1, groupId=spark-kafka-source-936e407b-2088-489d-8598-4fa287ce9ae7--1600603854-driver-0] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
24/02/21 10:16:11 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-936e407b-2088-489d-8598-4fa287ce9ae7--1600603854-d

24/02/21 10:16:13 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-936e407b-2088-489d-8598-4fa287ce9ae7--1600603854-executor-2, groupId=spark-kafka-source-936e407b-2088-489d-8598-4fa287ce9ae7--1600603854-executor] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
24/02/21 10:16:13 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-936e407b-2088-489d-8598-4fa287ce9ae7--1600603854-executor-2, groupId=spark-kafka-source-936e407b-2088-489d-8598-4fa287ce9ae7--1600603854-executor] Bootstrap broker localhost:9092 (id: -1 rack: null) disconnected


-------------------------------------------
Batch: 0
-------------------------------------------
+-----------------+--------------------------------------------------+----------+
|name             |address                                           |created_at|
+-----------------+--------------------------------------------------+----------+
|Ian Vega         |279 Justin Mission\nNew Peter, WI 58432           |2020      |
|Laura Ellis      |23476 Susan Overpass Apt. 279\nJuliestad, CA 10572|2010      |
|Edward Bond      |3053 Joe Stream Apt. 031\nNorth Joshua, PA 66613  |1984      |
|Stephanie Jackson|902 Ho Plains Suite 238\nHarrisside, DE 98796     |1985      |
+-----------------+--------------------------------------------------+----------+

-------------------------------------------
Batch: 1
-------------------------------------------
+------------------+------------------------------------------------------------+----------+
|name              |address                             

In [12]:
write_query.stop()

# Mahesh End

In [7]:
#1.3 Structure Streaming DataFrame

def parse_ride_from_kafka_message(df_raw, schema):
    """ take a Spark Streaming df and parse value col based on <schema>, return streaming df cols in schema """
    assert df_raw.isStreaming is True, "DataFrame doesn't receive streaming data"

    df = df_raw.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

    # split attributes to nested array in one Column
    col = F.split(df['value'], ', ')

    # expand col to multiple top-level columns
    for idx, field in enumerate(schema):
        df = df.withColumn(field.name, col.getItem(idx).cast(field.dataType))
    return df.select([field.name for field in schema])

In [8]:
ride_schema = T.StructType(
    [T.StructField("name", T.StringType()),
     T.StructField('address', T.StringType()),
     T.StructField('created_at', T.IntegerType()),
     ])

In [9]:
df_rides = parse_ride_from_kafka_message(df_raw=df_kafka_raw, schema=ride_schema)

In [ ]:
df_rides.printSchema()

# Console Sink

In [13]:
#

def sink_console(df, output_mode: str = 'complete', processing_time: str = '4 seconds'):
    write_query = df.writeStream \
        .outputMode(output_mode) \
        .trigger(processingTime=processing_time) \
        .format("console") \
        .option("truncate", False) \
        .start()
    return write_query # pyspark.sql.streaming.StreamingQuery

In [14]:
write_query = sink_console(df_rides, output_mode='append')

24/02/21 07:28:39 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-1205500f-87bb-4ccb-8819-726607c04314. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/02/21 07:28:39 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
-------------------------------------------
Batch: 0
-------------------------------------------
+----+-------+----------+
|name|address|created_at|
+----+-------+----------+
+----+-------+----------+

24/02/21 07:31:30 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-da9e98cb-e5c5-4022-aea6-d3252542ea5f--1044337870-executor-3, groupId=spark-kafka-source-da9e98cb-e5c5-4022-aea6-d3252542ea5f--1044337870-executor] Connection to node -1 (localhost/127.

-------------------------------------------
Batch: 2
-------------------------------------------
+------------------------+-----------------------------------------------+----------+
|name                    |address                                        |created_at|
+------------------------+-----------------------------------------------+----------+
|{"name": "Matthew Lucas"|"address": "1704 Perkins Harbors\nSouth Cynthia|null      |
+------------------------+-----------------------------------------------+----------+

24/02/21 07:31:32 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-d50e44ef-17da-417b-9917-30e313c98ce0-674426846-executor-4, groupId=spark-kafka-source-d50e44ef-17da-417b-9917-30e313c98ce0-674426846-executor] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
24/02/21 07:31:32 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-d50e44ef-17da-417b-9917-30e313c98ce0-674426846-executor

-------------------------------------------
Batch: 1
-------------------------------------------
+------------------------+-----------------------------------------------+----------+
|name                    |address                                        |created_at|
+------------------------+-----------------------------------------------+----------+
|{"name": "Matthew Lucas"|"address": "1704 Perkins Harbors\nSouth Cynthia|null      |
+------------------------+-----------------------------------------------+----------+

-------------------------------------------
Batch: 3
-------------------------------------------
+--------------------------+------------------------------------------+----------+
|name                      |address                                   |created_at|
+--------------------------+------------------------------------------+----------+
|{"name": "James Hernandez"|"address": "465 Nixon Harbors\nDouglastown|null      |
+--------------------------+---------------

In [15]:
write_query.stop()

# Memory Sink

In [16]:
def sink_memory(df, query_name, query_template):
    write_query = df \
        .writeStream \
        .queryName(query_name) \
        .format('memory') \
        .start()
    query_str = query_template.format(table_name=query_name)
    query_results = spark.sql(query_str)
    return write_query, query_results

In [17]:
query_name = 'name_counts'
query_template = 'select count(distinct(name)) from {table_name}'
write_query, df_vendor_id_counts = sink_memory(df=df_rides, query_name=query_name, query_template=query_template)

24/02/21 07:40:35 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-44514cc4-9af4-498e-ba02-a4388623937e. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/02/21 07:40:35 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/02/21 07:40:35 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-233305f2-07a1-42e7-bc2b-f04ff4436f33--1250756507-driver-0-5, groupId=spark-kafka-source-233305f2-07a1-42e7-bc2b-f04ff4436f33--1250756507-driver-0] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
24/02/21 07:40:35 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-233305f2-07a1-42e7-bc2b-f04ff4436f33--1250756507-d

-------------------------------------------
Batch: 11
-------------------------------------------
+------------------+-----------------------------------------------------------+----------+
|name              |address                                                    |created_at|
+------------------+-----------------------------------------------------------+----------+
|{"name": "Don Lam"|"address": "368 Monica Islands Suite 414\nNorth Sandramouth|null      |
+------------------+-----------------------------------------------------------+----------+

-------------------------------------------
Batch: 12
-------------------------------------------
+-------------------------------+--------------------------------------------------------+----------+
|name                           |address                                                 |created_at|
+-------------------------------+--------------------------------------------------------+----------+
|{"name": "Cameron Carrillo"    |"add

-------------------------------------------
Batch: 13
-------------------------------------------
+---------------------+--------------------------------------------------------+----------+
|name                 |address                                                 |created_at|
+---------------------+--------------------------------------------------------+----------+
|{"name": "Brian Long"|"address": "14389 Patterson Way Apt. 808\nWest Christina|null      |
+---------------------+--------------------------------------------------------+----------+



-------------------------------------------
Batch: 14
-------------------------------------------
+---------------------+--------------------------------------------------------+----------+
|name                 |address                                                 |created_at|
+---------------------+--------------------------------------------------------+----------+
|{"name": "Eric Foley"|"address": "6687 Pamela Inlet Suite 121\nNew Jordanshire|null      |
+---------------------+--------------------------------------------------------+----------+

-------------------------------------------
Batch: 15
-------------------------------------------
+----------------------+-------------------------------------------+----------+
|name                  |address                                    |created_at|
+----------------------+-------------------------------------------+----------+
|{"name": "Kari Burton"|"address": "3623 Castro Turnpike\nWeissport|null      |
+----------------------

-------------------------------------------
Batch: 16
-------------------------------------------
+---------------------------+--------------------------------------------+----------+
|name                       |address                                     |created_at|
+---------------------------+--------------------------------------------+----------+
|{"name": "Emily Macdonald" |"address": "0799 David Gardens\nMartinezfort|null      |
|{"name": "Katherine Vargas"|"address": "USNV Norris\nFPO AE 11185"      |null      |
+---------------------------+--------------------------------------------+----------+



In [ ]:
write_query.stop()